In [80]:
"""
5 API requests per minute.
25 requests per day.
Retrieves 100 previous results.
"""
import requests
import pandas as pd
import scipy.stats as sts
import numpy as np
import os
from datetime import datetime
from prophet import Prophet
API_KEY = 'YOUR_KEY_HERE'

# STOCK SYMBOL
symbol = "TSLA"
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_KEY}'
data = requests.get(url).json()

df = pd.DataFrame(data['Time Series (Daily)']).T
df = df.apply(pd.to_numeric, errors='ignore')
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'Date', '1. open':'Open', '2. high':'High', 
                        '3. low': 'Low','4. close':'Close','5. volume':'Volume'})

df['Company'] = symbol
df.head()

,Date,Open,High,Low,Close,Volume,Company
0,2024-01-09,238.11,238.9646,232.0400,234.96,96705664,TSLA
1,2024-01-08,236.14,241.2500,235.3000,240.45,85166580,TSLA
2,2024-01-05,236.86,240.1196,234.9001,237.49,92240035,TSLA
3,2024-01-04,239.25,242.7000,237.7300,237.93,102629283,TSLA
4,2024-01-03,244.98,245.6800,236.3200,238.45,121082599,TSLA


In [81]:
prophet_df = df[['Date','Close']].sort_values("Date", ascending=True).reset_index(drop=True)
prophet_df.columns = ['ds','y']
prophet_df = prophet_df.dropna()

m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=30, freq='B')
forecast = m.predict(future)
close_trend_30days = forecast[['ds','trend','yhat', 'yhat_lower', 'yhat_upper']].iloc[-30:,:]

16:03:28 - cmdstanpy - INFO - Chain [1] start processing
16:03:28 - cmdstanpy - INFO - Chain [1] done processing


In [82]:
prophet_df = df[['Date','High']].sort_values("Date", ascending=True).reset_index(drop=True)
prophet_df.columns = ['ds','y']
prophet_df = prophet_df.dropna()

m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=10, freq='B')
forecast = m.predict(future)
high_trend = forecast[['ds','trend','yhat', 'yhat_lower', 'yhat_upper']].iloc[-10:,:]

prophet_df = df[['Date','Low']].sort_values("Date", ascending=True).reset_index(drop=True)
prophet_df.columns = ['ds','y']
prophet_df = prophet_df.dropna()

m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=10, freq='B')
forecast = m.predict(future)
low_trend = forecast[['ds','trend','yhat', 'yhat_lower', 'yhat_upper']].iloc[-10:,:]

prophet_df = df[['Date','Close']].sort_values("Date", ascending=True).reset_index(drop=True)
prophet_df.columns = ['ds','y']
prophet_df = prophet_df.dropna()

m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=10, freq='B')
forecast = m.predict(future)
close_trend = forecast[['ds','trend','yhat', 'yhat_lower', 'yhat_upper']].iloc[-10:,:]

16:03:31 - cmdstanpy - INFO - Chain [1] start processing
16:03:31 - cmdstanpy - INFO - Chain [1] done processing
16:03:31 - cmdstanpy - INFO - Chain [1] start processing
16:03:32 - cmdstanpy - INFO - Chain [1] done processing
16:03:32 - cmdstanpy - INFO - Chain [1] start processing
16:03:32 - cmdstanpy - INFO - Chain [1] done processing


In [83]:
# TEST TO SEE IF CLOSING IS TRENDING UP OR DOWN LONG TERM
'up' if close_trend_30days['yhat'].iloc[-1] > close_trend_30days['yhat'].iloc[0] else 'down'

'up'